OpenAQ

In [ ]:
API_KEY = "94d98cbdd0c42919e1017aa7c619b0ec47fa75c988e232eafb8e8f8e01c3584e"

In [ ]:
!pip install openaq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from openaq import OpenAQ

import time
from datetime import datetime, timedelta, timezone
from functools import reduce

import warnings
warnings.simplefilter('ignore')

In [ ]:
api = OpenAQ(API_KEY)

OpenAQ dùng country code theo chuẩn ISO 3166-1 alpha 2

Tìm ID của Việt Nam bằng country code = "VN"


In [ ]:
countries = api.countries.list()
countries_dict = countries.dict()

for country in countries_dict['results']:
    if country['code'] == 'VN':
        vn_country_id = country['id']
        print(f"Country ID for Vietnam: {vn_country_id}")
        break

Country ID for Vietnam: 56


In [ ]:
vn_info = api.countries.get(countries_id=vn_country_id)
vn_info_dict = vn_info.dict()
vn_info_dict

{'headers': {'x_ratelimit_limit': 60,
  'x_ratelimit_remaining': 59,
  'x_ratelimit_used': 1,
  'x_ratelimit_reset': 60},
 'meta': {'name': 'openaq-api',
  'website': '/',
  'page': 1,
  'limit': 100,
  'found': 1},
 'results': [{'id': 56,
   'code': 'VN',
   'name': 'Vietnam',
   'datetime_first': '2016-01-30T01:00:00Z',
   'datetime_last': '2025-05-18T05:00:00Z',
   'parameters': [{'id': 1,
     'name': 'pm10',
     'units': 'µg/m³',
     'display_name': None},
    {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'display_name': None},
    {'id': 3, 'name': 'o3', 'units': 'µg/m³', 'display_name': None},
    {'id': 4, 'name': 'co', 'units': 'µg/m³', 'display_name': None},
    {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'display_name': None},
    {'id': 6, 'name': 'so2', 'units': 'µg/m³', 'display_name': None},
    {'id': 19, 'name': 'pm1', 'units': 'µg/m³', 'display_name': None},
    {'id': 98, 'name': 'relativehumidity', 'units': '%', 'display_name': None},
    {'id': 100, 'name': 'tempera

Liệt kê locations ở Việt Nam

In [ ]:
vn_locations = api.locations.list(countries_id=56)
vn_locations_dict = vn_locations.dict()
vn_locations_dict

{'headers': {'x_ratelimit_limit': 60,
  'x_ratelimit_remaining': 59,
  'x_ratelimit_used': 1,
  'x_ratelimit_reset': 60},
 'meta': {'name': 'openaq-api',
  'website': '/',
  'page': 1,
  'limit': 100,
  'found': 47},
 'results': [{'id': 18,
   'name': 'SPARTAN - Vietnam Acad. Sci.',
   'locality': None,
   'timezone': 'Asia/Bangkok',
   'country': {'id': 56, 'code': 'VN', 'name': 'Vietnam'},
   'owner': {'id': 4, 'name': 'Unknown Governmental Organization'},
   'provider': {'id': 226, 'name': 'Spartan'},
   'is_mobile': False,
   'is_monitor': True,
   'instruments': [{'id': 2, 'name': 'Government Monitor'}],
   'sensors': [{'id': 37,
     'name': 'pm25 µg/m³',
     'parameter': {'id': 2,
      'name': 'pm25',
      'units': 'µg/m³',
      'display_name': 'PM2.5'}}],
   'coordinates': {'latitude': 21.048, 'longitude': 105.8},
   'bounds': [105.8, 21.048, 105.8, 21.048],
   'distance': None,
   'datetime_first': None,
   'datetime_last': None},
  {'id': 2446,
   'name': 'US Diplomatic P

In [ ]:
from datetime import datetime

longest_duration = None
longest_location = None

for loc in vn_locations_dict['results']:
    dt_first = loc.get('datetime_first')
    dt_last = loc.get('datetime_last')

    # Bỏ qua nếu thiếu dữ liệu
    if not dt_first or not dt_last:
        continue

    try:
        t1 = datetime.strptime(dt_first['utc'], "%Y-%m-%dT%H:%M:%SZ")
        t2 = datetime.strptime(dt_last['utc'], "%Y-%m-%dT%H:%M:%SZ")
        duration = t2 - t1
    except Exception as e:
        print(f"Lỗi khi xử lý location {loc['id']}: {e}")
        continue

    if longest_duration is None or duration > longest_duration:
        longest_duration = duration
        longest_location = loc

# In ra kết quả
if longest_location:
    print("Location có thời gian đo dài nhất:")
    print(f"ID: {longest_location['id']}")
    print(f"Name: {longest_location['name']}")
    print(f"From: {longest_location['datetime_first']['utc']}")
    print(f"To:   {longest_location['datetime_last']['utc']}")
    print(f"Duration: {longest_duration}")
else:
    print("⚠️ Không có location nào có đủ dữ liệu datetime_first và datetime_last.")


Location có thời gian đo dài nhất:
ID: 7441
Name: Hanoi
From: 2016-11-09T18:00:00Z
To:   2025-04-09T15:00:00Z
Duration: 3072 days, 21:00:00


In [ ]:
from datetime import datetime

locations_with_duration = []

for loc in vn_locations_dict['results']:
    dt_first = loc.get('datetime_first')
    dt_last = loc.get('datetime_last')

    # Bỏ qua nếu thiếu dữ liệu
    if not dt_first or not dt_last:
        continue

    try:
        t1 = datetime.strptime(dt_first['utc'], "%Y-%m-%dT%H:%M:%SZ")
        t2 = datetime.strptime(dt_last['utc'], "%Y-%m-%dT%H:%M:%SZ")
        duration_hours = (t2 - t1).total_seconds() / 3600
    except Exception as e:
        print(f"Lỗi khi xử lý location {loc['id']}: {e}")
        continue

    num_sensors = len(loc.get('sensors', []))
    locations_with_duration.append({
        'id': loc['id'],
        'name': loc['name'],
        'duration': duration_hours,
        'datetime_from': dt_first['utc'],
        'datetime_to': dt_last['utc'],
        'num_sensors': num_sensors
    })

# Sắp xếp theo thời gian giảm dần, rồi theo số lượng sensor giảm dần
sorted_locations = sorted(
    locations_with_duration,
    key=lambda x: (x['duration'], x['num_sensors']),
    reverse=True
)

# In ra kết quả
for loc in sorted_locations:
    print(f"{loc['name']} (ID: {loc['id']})")
    print(f"   Duration: {loc['duration']}, Sensors: {loc['num_sensors']}")
    print(f"   From: {loc['datetime_from']}  To: {loc['datetime_to']}\n")

Hanoi (ID: 7441)
   Duration: 73749.0, Sensors: 1
   From: 2016-11-09T18:00:00Z  To: 2025-04-09T15:00:00Z

US Diplomatic Post: Ho Chi Minh City (ID: 7440)
   Duration: 72883.0, Sensors: 1
   From: 2016-11-09T17:00:00Z  To: 2025-03-04T12:00:00Z

Đào Duy Từ (ID: 2161296)
   Duration: 11734.0, Sensors: 4
   From: 2024-01-15T07:00:00Z  To: 2025-05-18T05:00:00Z

An Khánh (ID: 2161290)
   Duration: 11399.0, Sensors: 4
   From: 2024-01-29T06:00:00Z  To: 2025-05-18T05:00:00Z

Vân Hà (ID: 2161320)
   Duration: 11390.0, Sensors: 4
   From: 2024-01-29T15:00:00Z  To: 2025-05-18T05:00:00Z

Số 46, phố Lưu Quang Vũ (ID: 2161292)
   Duration: 11389.0, Sensors: 6
   From: 2024-01-29T16:00:00Z  To: 2025-05-18T05:00:00Z

Minh Khai - Bắc Từ Liêm (ID: 2161306)
   Duration: 11389.0, Sensors: 6
   From: 2024-01-29T16:00:00Z  To: 2025-05-18T05:00:00Z

Sóc Sơn (ID: 2161312)
   Duration: 9954.0, Sensors: 4
   From: 2024-01-29T15:00:00Z  To: 2025-03-19T09:00:00Z

Xuân Mai (ID: 2161323)
   Duration: 9907.0, Senso

ID: 2161292
ID: 2161306

In [ ]:
for location in vn_locations_dict['results']:
    if location['id'] == 2161290:
        btl_location = location
        break

In [ ]:
sensors = api.locations.sensors(locations_id = 2161290)
sensors_dict = sensors.dict()
sensors_dict

{'headers': {'x_ratelimit_limit': 60,
  'x_ratelimit_remaining': 59,
  'x_ratelimit_used': 1,
  'x_ratelimit_reset': 60},
 'meta': {'name': 'openaq-api',
  'website': '/',
  'page': 1,
  'limit': 100,
  'found': 4},
 'results': [{'id': 7771981,
   'name': 'co µg/m³',
   'parameter': {'id': 4,
    'name': 'co',
    'units': 'µg/m³',
    'displayName': 'CO mass'},
   'datetime_first': {'utc': '2024-01-29T06:00:00Z',
    'local': '2024-01-29T13:00:00+07:00'},
   'datetime_last': {'utc': '2025-05-18T09:00:00Z',
    'local': '2025-05-18T16:00:00+07:00'},
   'coverage': {'expectedCount': 1,
    'expectedInterval': '01:00:00',
    'observedCount': 8876,
    'observedInterval': '8876:00:00',
    'percentComplete': 887600.0,
    'percentCoverage': 887600.0,
    'datetimeFrom': {'utc': '2024-01-29T06:00:00Z',
     'local': '2024-01-29T13:00:00+07:00'},
    'datetimeTo': {'utc': '2025-05-18T09:00:00Z',
     'local': '2025-05-18T16:00:00+07:00'}},
   'latest': {'datetime': {'utc': '2025-05-18T09:0

In [ ]:
sensor_name_map = {
    sensor['id']: sensor['name']
    for sensor in sensors_dict['results']
}
sensor_name_map

{7771981: 'co µg/m³',
 7772012: 'pm25 µg/m³',
 7772033: 'no2 µg/m³',
 7772095: 'pm10 µg/m³'}

In [ ]:
dataframes = []

for sensor in sensors_dict['results']:
    sensor_id = sensor['id']
    sensor_name = sensor_name_map[sensor_id]

    datetime_first = datetime.strptime(sensor['datetime_first']['utc'], "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)
    datetime_last = datetime.strptime(sensor['datetime_last']['utc'], "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)

    all_measurements = []
    page = 1
    limit = 1000

    while True:
        try:
            response = api.measurements.list(
                sensors_id=sensor_id,
                data='hours',
                datetime_from=datetime_first,
                datetime_to=datetime_last,
                page=page,
                limit=limit
            )
            response = response.dict()
        except Exception as e:
            print(f"Error for sensor {sensor_id} page {page}: {e}")
            break

        results = response.get('results', [])
        if not results:
            break

        all_measurements.extend(results)
        print(f"Sensor {sensor_name} - Page {page}: {len(results)} records")

        if len(results) < limit:
            break

        page += 1
        time.sleep(1)

    if not all_measurements:
        print(f"No data for sensor {sensor_name}")
        continue

    print(all_measurements)

    # Tạo DataFrame cho sensor từ các trường phù hợp với cấu trúc response mới
    df = pd.DataFrame([{
        'datetimeFrom_utc': r['period']['datetime_from']['utc'],
        'datetimeFrom_local': r['period']['datetime_from']['local'],
        'datetimeTo_utc': r['period']['datetime_to']['utc'],
        'datetimeTo_local': r['period']['datetime_to']['local'],
        sensor_name: r['value']
    } for r in all_measurements])

    dataframes.append(df)

# Gộp các DataFrame theo datetime
df_merged = reduce(lambda left, right: pd.merge(left, right, on=['datetimeFrom_utc', 'datetimeTo_utc', 'datetimeFrom_local', 'datetimeTo_local'], how='outer'), dataframes)

# Sắp xếp theo thời gian
df_merged = df_merged.sort_values(by='datetimeFrom_utc')

# Lưu ra file
filename = "ankhanh_measurements.csv"
df_merged.to_csv(filename, index=False)
print(f"Saved data to {filename}")

Output hidden; open in https://colab.research.google.com to view.

# **Pipeline**

In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta, timezone
from openaq import OpenAQ
from functools import reduce
import holidays

class OpenAQProcessor:
    def __init__(self, API_KEY):
        self.api = OpenAQ(API_KEY)

    def get_data_for_prediction(self, location_id=2161290):
        # Vì offset là 24h nên khung thời gian có thể dự đoán sẽ từ thời điểm hiện tại đến 48 giờ sau.
        sensors = self.api.locations.sensors(locations_id = 2161290)
        sensors_dict = sensors.dict()

        sensor_name_map = {
            sensor['id']: sensor['name']
            for sensor in sensors_dict['results']
        }

        dataframes = []


        for sensor in sensors_dict['results']:
            sensor_id = sensor['id']
            sensor_name = sensor_name_map[sensor_id]

            # Thời gian hiện tại (UTC)
            datetime_last = datetime.now(timezone.utc)

            # Thời gian hiện tại trừ đi 96 giờ
            datetime_first = datetime_last - timedelta(hours=96)

            # Định dạng lại chuỗi thời gian nếu cần thiết để phù hợp với API OpenAQ
            # Ví dụ: '2023-10-27T10:00:00Z'
            datetime_first_str = datetime_first.strftime("%Y-%m-%dT%H:%M:%SZ")
            datetime_last_str = datetime_last.strftime("%Y-%m-%dT%H:%M:%SZ")

            print(f"datetime_first: {datetime_first_str}")
            print(f"datetime_last: {datetime_last_str}")

            all_measurements = []
            page = 1
            limit = 1000        # Cần 96h để dự đoán 48h kế tiếp

            try:
                response = self.api.measurements.list(
                    sensors_id=sensor_id,
                    data='hours',
                    datetime_from=datetime_first,
                    datetime_to=datetime_last,
                    page=page,
                    limit=limit
                )
                response = response.dict()
            except Exception as e:
                print(f"Error for sensor {sensor_id} page {page}: {e}")
                break

            results = response.get('results', [])

            # Tạo DataFrame cho sensor từ các trường phù hợp với cấu trúc response mới
            df = pd.DataFrame([{
                'datetimeFrom_local': r['period']['datetime_from']['local'],
                sensor_name: r['value']
            } for r in results])

            dataframes.append(df)

        # Gộp các DataFrame theo datetime
        df_merged = reduce(lambda left, right: pd.merge(left, right, on=['datetimeFrom_local'], how='outer'), dataframes)

        # Sắp xếp theo thời gian
        df_merged = df_merged.sort_values(by='datetimeFrom_local')

        return df_merged


    def preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        # Đổi tên cột: loại bỏ đơn vị
        df.rename(columns={
            "co µg/m³": "co",
            "pm25 µg/m³": "pm25",
            "no2 µg/m³": "no2",
            "pm10 µg/m³": "pm10"
        }, inplace=True)
        df.drop(columns=['pm10'], inplace=True)
        df['datetimeFrom_local'] = pd.to_datetime(df['datetimeFrom_local'])
        df = df.set_index('datetimeFrom_local')

        # Tạo lại dãy thời gian liên tục
        full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='1h')
        df = df.reindex(full_index)

        # Xử lý missing values dùng forward fill
        df.fillna(method='ffill', inplace=True)

        # Các cột cần transform
        cols_to_transform = ["co", "pm25", "no2"]

        # Apply log transform ONLY to the specified columns
        transformed_df = df.copy()

        # Apply log1p transform only to the specified columns
        transformed_df[cols_to_transform] = np.log1p(transformed_df[cols_to_transform])

        # Replace the original columns with log-transformed values in the main DataFrame
        df[cols_to_transform] = transformed_df[cols_to_transform]

        # Thêm time-based features
        vn_holidays = holidays.Vietnam()
        df['hour'] = df.index.hour
        df['dayofweek'] = df.index.dayofweek
        df['is_holiday'] = df.index.normalize().isin(vn_holidays)

        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
        df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
        df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
        df['is_holiday'] = df['is_holiday'].astype(int)

        df.drop(columns=['hour', 'dayofweek'], inplace=True)

        return df



In [ ]:
processor = OpenAQProcessor(API_KEY)
df = processor.get_data_for_prediction()
df_preprocessed = processor.preprocess(df)
df_preprocessed

datetime_first: 2025-05-18T17:12:57Z
datetime_last: 2025-05-20T17:12:57Z
datetime_first: 2025-05-18T17:12:57Z
datetime_last: 2025-05-20T17:12:57Z
datetime_first: 2025-05-18T17:12:57Z
datetime_last: 2025-05-20T17:12:57Z
datetime_first: 2025-05-18T17:12:58Z
datetime_last: 2025-05-20T17:12:58Z


<ipython-input-22-2d178a6fc2c0>:99: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


,co,pm25,no2,is_holiday,hour_sin,hour_cos,dayofweek_sin,dayofweek_cos
2025-05-19 00:00:00+07:00,6.563856,3.144152,3.937691,0,0.000000e+00,1.000000e+00,0.000000,1.00000
2025-05-19 01:00:00+07:00,4.152613,1.704748,3.246491,0,2.588190e-01,9.659258e-01,0.000000,1.00000
2025-05-19 02:00:00+07:00,4.286341,1.704748,3.446808,0,5.000000e-01,8.660254e-01,0.000000,1.00000
2025-05-19 03:00:00+07:00,4.172848,1.481605,3.284664,0,7.071068e-01,7.071068e-01,0.000000,1.00000
2025-05-19 04:00:00+07:00,4.354141,1.609438,3.034953,0,8.660254e-01,5.000000e-01,0.000000,1.00000
2025-05-19 05:00:00+07:00,4.919981,1.974081,3.086487,0,9.659258e-01,2.588190e-01,0.000000,1.00000
2025-05-19 06:00:00+07:00,5.897154,2.517696,3.182212,0,1.000000e+00,6.123234e-17,0.000000,1.00000
2025-05-19 07:00:00+07:00,6.728629,2.850707,3.465736,0,9.659258e-01,-2.588190e-01,0.000000,1.00000
2025-05-19 08:00:00+07:00,6.898715,3.063391,3.517498,0,8.660254e-01,-5.000000e-01,0.000000,1.00000
2025-05-19 09:00:00+07:00,6.994850,3.234749,3.605498,0,7.071068e-01,-7.071068e-01,0.000000,1.00000
